# Demo Notebook for Creating an Analysis of Reboot Motion Time Series Data

__[CoLab Notebook Link](https://githubtocolab.com/RebootMotion/reboot-toolkit/blob/main/examples/RebootMotionAnalysisGeneralMovements.ipynb)__

Run the cells in order, making sure to enter AWS credentials in the cell when prompted

In [ ]:
#@title Install Python Package

!pip install git+https://github.com/RebootMotion/reboot-toolkit.git@v2.10.2#egg=reboot_toolkit > /dev/null
!echo "Done Installing"

In [ ]:
#@title Import Python Libraries

import awswrangler as wr
import reboot_toolkit as rtk
import matplotlib.pyplot as plt
import os
import pandas as pd

from reboot_toolkit import S3Metadata, MocapType, MovementType, Handedness, FileType, PlayerMetadata
from IPython.display import display

In [ ]:
#@title AWS Credentials

# Upload your Organization's .env file to the local file system, per https://pypi.org/project/python-dotenv/
# OR input your credentials string generated by the Reboot Dashboard

boto3_session = rtk.setup_aws()

In [ ]:
#@title User Input - No code changes required below this section, just enter information in forms

# Update the below info to match your desired analysis information
# Common changes you might want to make:

# To analyze files captured with Reboot's single camera solution
# set mocap_types=[MocapType.REBOOT_MOTION]

# To analyze general movements, set the appropriate movement type:
# Options: VERTICAL_JUMPING, SQUATTING, LUNGING, LEG_RAISING
# set movement_type=MovementType.VERTICAL_JUMPING

# If handessness doesn't apply,
# set handedness=Handedness.NONE

# To analyze data from the inverse kinematics files,
# set file_type=FileType.INVERSE_KINEMATICS

# See https://docs.rebootmotion.com/ for all available file types and the data in each
mocap_types = [MocapType.REBOOT_MOTION]
movement_type = MovementType.VERTICAL_JUMPING
handedness = Handedness.NONE
file_type = FileType.INVERSE_KINEMATICS

# Update the label to whatever you'd like to be displayed in the visuals
primary_segment_label = 'Primary Segment'

In [ ]:
#@title Set S3 File Info

s3_metadata = S3Metadata(
    org_id=os.environ['ORG_ID'],
    mocap_types=mocap_types,
    movement_type=movement_type,
    handedness=handedness,
    file_type=file_type,
)

s3_df = rtk.download_s3_summary_df(s3_metadata)

In [ ]:
#@title Optional Look Up Player ID by Name

name_to_look_up = "Jacob deGrom"

rtk.find_player_matches(s3_df, name_to_look_up, match_threshold=50., max_results=5)

In [ ]:
#@title Display the Interface for Selecting the Primary Data Segment to Analyze

# Run this cell to display the dropdown menus and reset all options to NULL
primary_segment_widget = rtk.create_interactive_widget(s3_df)
display(primary_segment_widget)

In [ ]:
#@title Set Primary Analysis Segment Info

primary_segment_data = primary_segment_widget.children[1].result
primary_analysis_segment = PlayerMetadata(
    org_player_ids=primary_segment_data["org_player_ids"],
    session_dates=primary_segment_data["session_dates"],
    session_nums=primary_segment_data["session_nums"],
    session_date_start=primary_segment_data["session_date_start"],
    session_date_end=primary_segment_data["session_date_end"],
    year=primary_segment_data["year"],
    org_movement_id=None, # set the play GUID for the skeleton animation; None defaults to the first play
    s3_metadata=s3_metadata,
)

primary_segment_summary_df = rtk.filter_s3_summary_df(primary_analysis_segment, s3_df)

# Common Issue:
# If no data files are returned here,
# check that the segment selection widget and the S3 File Info above are set correctly,
# also if the cells were updated after running once, check that they were run again with any new selections.

# List all Available S3 data for the Primary Analysis Segment
available_s3_keys = rtk.list_available_s3_keys(os.environ['ORG_ID'], primary_segment_summary_df)
# Load the Primary Analysis Segment into an Analysis Dictionary with Mean and Standard Dev DataFrames
primary_segment_data_df = rtk.load_games_to_df_from_s3_paths(primary_segment_summary_df['s3_path_delivery'].tolist())

# Common Issue:
# Missing element: data integrity issues in parsing. Majority of data is still likely fine.
# Common metrics to filter by are: start_speed, end_speed, spin_rate, spin_direction, zone, pitch_type

In [ ]:
#@title Load data into format for processing

primary_segment_dict = rtk.load_data_into_analysis_dict(primary_analysis_segment, primary_segment_data_df, segment_label=primary_segment_label)

In [ ]:
# Movement Files in Dataframe
movement_uids =primary_segment_data_df['org_movement_id'].unique()
print(list(movement_uids))

In [ ]:
#@title Optional - Create Simple Comparison Plots

# Available time options for the x_column include: 'time_from_max_hand', 'norm_time', 'rel_frame', 'time'
# Note if analyzing basketball, "time_from..." column is 'time_from_max_height'
x_column = 'time'
#x_column = 'norm_time'
#x_column = 'rel_frame'

# Below are examples of possible joint angles of interest for a vertical jump
# feel free to update to any available data from the INVERSE KINEMATICS file type here https://docs.rebootmotion.com/,
# or if your file type is MOMENTUM_ENERGY, you can update "y_columns" to columns from that file type
y_columns = [
    'right_hip_flex', 'left_hip_flex',
    'right_knee', 'left_knee'
]

# Set the y-axis label to whatever is appropriate for the y-columns above
y_axis_label = "joint angle (deg)"

mpl_figs = []

for y_column in y_columns:

    mpl_fig = plt.figure()

    for org_movement_id in movement_uids:
        movement_df = (
            primary_segment_data_df.loc[primary_segment_data_df["org_movement_id"] == org_movement_id]
            .copy()
            .reset_index(drop=True)
        )

        plt.plot(movement_df[x_column], movement_df[y_column], label=org_movement_id)

    plt.ylabel(y_axis_label)
    # plt.ylim([-180, 180])  # uncomment to limit the height of the y-axis

    plt.xlabel(x_column)
    # plt.xlim([-1, 0.1])  # uncomment to limit the width of the x-axis

    plt.title(y_column)

    plt.legend()

    plt.grid()

    plt.show()

    mpl_figs.append(mpl_fig)

In [ ]:
#@title Optional - Save Plots to a PDF

from matplotlib.backends.backend_pdf import PdfPages

pdf_file_name = 'analysis.pdf'

pdf_analysis = PdfPages(pdf_file_name)

for mpl_fig in mpl_figs:

    pdf_analysis.savefig(mpl_fig)

pdf_analysis.close()

print('Saved plots to', pdf_file_name)